In [ ]:
from flask import Flask, request, jsonify, render_template
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)
def load_qa_pairs(filepath):
    with open(filepath, 'r', errors='ignore') as file:
        raw_data = file.read().strip().split('\n')
    qa_pairs = {}
    question = None
    for line in raw_data:
        if line.startswith('Q'):
            question = line[2:].strip()  # Extract question part
        elif line.startswith('A') and question is not None:
            answer = line[2:].strip()    # Extract answer part
            qa_pairs[question] = answer  # Store Q&A in a dictionary
            question = None
    return qa_pairs

qa_pairs = load_qa_pairs('data.txt')


nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

def preprocess_text(text):
    return ' '.join(LemNormalize(text))

greet_inputs = ('hello', 'hi', 'hey', 'wassup')
greet_response = 'Hey! I am Eco Bot. How can I help you with something related to sustainability?'

def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return greet_response
    return None


def find_best_answer(user_input, qa_pairs):
    questions = list(qa_pairs.keys())
    tfidf_vectorizer = TfidfVectorizer(tokenizer=LemNormalize)
    all_texts = [preprocess_text(q) for q in questions] + [preprocess_text(user_input)]
    tfidf_matrix = tfidf_vectorizer.fit_transform(all_texts)
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    similar_idx = np.argmax(cosine_similarities)
    if cosine_similarities[0, similar_idx] > 0.3:
        return qa_pairs[questions[similar_idx]]
    else:
        return "I am Sorry! I can't understand you, can you please repeat your question?"

@app.route('/')
def home():
    return render_template('bot.html')  # Serve bot.html from the templates folder


@app.route('/get', methods=['POST'])
def get_response():
    user_input = request.form['msg']
    response = greet(user_input) or find_best_answer(user_input, qa_pairs)
    return jsonify({'response': response})

if __name__ == "__main__":
    app.run(debug=True, use_reloader = False)


 * Serving Flask app '__main__'
 * Debug mode: on


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Sep/2024 22:30:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2024 22:30:53] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2024 22:37:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2024 22:37:58] "POST /get HTTP/1.1" 200 -
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
127.0.0.1 - - [12/Sep/2024 22:38:08] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2024 22:38:14] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2024 22:38:27] "POST /get HTTP/1.1" 200 -
127.